In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as mtp
import numpy as np
%matplotlib inline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# step: represents a unit of time where 1 step equals 1 hour
# type: type of online transaction
# amount: the amount of the transaction
# nameOrig: customer starting the transaction
# oldbalanceOrg: balance before the transaction
# newbalanceOrig: balance after the transaction
# nameDest: recipient of the transaction
# oldbalanceDest: initial balance of recipient before the transaction
# newbalanceDest: the new balance of recipient after the transaction
# isFraud: fraud transaction

In [3]:
df= pd.read_csv('PS_20174392719_1491204439457_log.csv')
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [4]:
df_sampled=pd.DataFrame(df.head(100000))
df_sampled

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,10,PAYMENT,4020.66,C1410794718,159929.0,155908.34,M1257036576,0.00,0.00,0,0
99996,10,PAYMENT,18345.49,C744303677,6206.0,0.00,M1785344556,0.00,0.00,0,0
99997,10,CASH_IN,183774.91,C104331851,39173.0,222947.91,C36392889,54925.05,0.00,0,0
99998,10,CASH_OUT,82237.17,C707662966,6031.0,0.00,C1553004158,592635.66,799140.46,0,0


In [5]:
label_encoder =preprocessing.LabelEncoder()
df['type']=label_encoder.fit_transform(df['type'])
df['nameOrig']=label_encoder.fit_transform(df['nameOrig'])
df['nameDest']=label_encoder.fit_transform(df['nameDest'])

In [6]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,757869,170136.0,160296.36,1662094,0.0,0.0,0,0
1,1,3,1864.28,2188998,21249.0,19384.72,1733924,0.0,0.0,0,0
2,1,4,181.00,1002156,181.0,0.00,439685,0.0,0.0,1,0
3,1,1,181.00,5828262,181.0,0.00,391696,21182.0,0.0,1,0
4,1,3,11668.14,3445981,41554.0,29885.86,828919,0.0,0.0,0,0


In [7]:
scaler = preprocessing.StandardScaler()

In [8]:
scaler.fit(df.drop('isFraud',axis=1))
scaled_features = scaler.transform(df.drop('isFraud',axis=1))

In [9]:
X = pd.DataFrame(scaled_features,columns=df.columns[:-1])
X

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,-1.703042,0.952399,-0.281560,-1.318825,-0.229810,-0.237622,1.220490,-0.323814,-0.333411,-0.001586
1,-1.703042,0.952399,-0.294767,-0.538520,-0.281359,-0.285812,1.316232,-0.323814,-0.333411,-0.001586
2,-1.703042,1.693076,-0.297555,-1.185630,-0.288654,-0.292442,-0.408856,-0.323814,-0.333411,-0.001586
3,-1.703042,-0.528954,-0.297555,1.445742,-0.288654,-0.292442,-0.472820,-0.317582,-0.333411,-0.001586
4,-1.703042,0.952399,-0.278532,0.146834,-0.274329,-0.282221,0.109953,-0.323814,-0.333411,-0.001586
...,...,...,...,...,...,...,...,...,...,...
6362615,3.510123,-0.528954,0.264665,1.349554,-0.171108,-0.292442,-0.320647,-0.323814,-0.240959,-0.001586
6362616,3.510123,1.693076,10.153953,-0.784815,1.896491,-0.292442,-0.647092,-0.323814,-0.333411,-0.001586
6362617,3.510123,-0.528954,10.153953,-1.440909,1.896491,-0.292442,-0.850659,-0.303665,1.403027,-0.001586
6362618,3.510123,1.693076,1.109765,-0.503661,0.005581,-0.292442,-0.568766,-0.323814,-0.333411,-0.001586


In [10]:
X.rename(columns = {'isFraud':'isFlaggedFraud'}, inplace = True)

In [11]:
X

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,-1.703042,0.952399,-0.281560,-1.318825,-0.229810,-0.237622,1.220490,-0.323814,-0.333411,-0.001586
1,-1.703042,0.952399,-0.294767,-0.538520,-0.281359,-0.285812,1.316232,-0.323814,-0.333411,-0.001586
2,-1.703042,1.693076,-0.297555,-1.185630,-0.288654,-0.292442,-0.408856,-0.323814,-0.333411,-0.001586
3,-1.703042,-0.528954,-0.297555,1.445742,-0.288654,-0.292442,-0.472820,-0.317582,-0.333411,-0.001586
4,-1.703042,0.952399,-0.278532,0.146834,-0.274329,-0.282221,0.109953,-0.323814,-0.333411,-0.001586
...,...,...,...,...,...,...,...,...,...,...
6362615,3.510123,-0.528954,0.264665,1.349554,-0.171108,-0.292442,-0.320647,-0.323814,-0.240959,-0.001586
6362616,3.510123,1.693076,10.153953,-0.784815,1.896491,-0.292442,-0.647092,-0.323814,-0.333411,-0.001586
6362617,3.510123,-0.528954,10.153953,-1.440909,1.896491,-0.292442,-0.850659,-0.303665,1.403027,-0.001586
6362618,3.510123,1.693076,1.109765,-0.503661,0.005581,-0.292442,-0.568766,-0.323814,-0.333411,-0.001586


In [12]:
Y=df['isFraud']
Y

0          0
1          0
2          1
3          1
4          0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 6362620, dtype: int64

In [13]:
##datasplt
X_train, X_test, y_train, y_test = train_test_split(X,Y
                                                    ,test_size=0.40 )
X_cv,X_test1,y_cv,y_test1=train_test_split(X_test,y_test
                                                    ,test_size=0.50)



In [14]:
X_train.shape,y_train.shape

((3817572, 10), (3817572,))

In [15]:
X_test1.shape,y_test1.shape

((1272524, 10), (1272524,))

In [16]:
X_cv.shape,y_cv.shape

((1272524, 10), (1272524,))

In [18]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression  
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
model=linear_model.LogisticRegression(penalty='none')
model.fit(X_train,y_train)
Y_predict_test1=model.predict(X_test1)
Y_predict_cv=model.predict(X_cv)
print('Mean squared error for test data (MSE): %.10f'

      % mean_squared_error(y_test1, Y_predict_test1))

print('Mean squared error for cv data (MSE): %.10f'

      % mean_squared_error(y_cv, Y_predict_cv))




Mean squared error for test data (MSE): 0.0006883957
Mean squared error for cv data (MSE): 0.0006679638


In [19]:
print ("Accuracy : ", accuracy_score(y_cv, Y_predict_cv))
print ("Accuracy : ", accuracy_score(y_test1,Y_predict_test1))

Accuracy :  0.9993320361737774
Accuracy :  0.999311604339093


In [21]:
conf=confusion_matrix(y_test1,Y_predict_test1)
conf


array([[1270769,      89],
       [    787,     879]], dtype=int64)

# K-folding 

In [22]:
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [23]:
# prepare the cross-validation procedure
cv = KFold(n_splits=20, random_state=0, shuffle=True)
# evaluate model
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.999 (0.000)


# Stratified-KFold



In [24]:
from sklearn.model_selection import StratifiedKFold
# prepare the cross-validation procedure
cv_stratified= StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
# evaluate model
scores_stratified = cross_val_score(model, X, Y, scoring='accuracy', cv=cv_stratified, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores_stratified), std(scores_stratified)))


Accuracy: 0.999 (0.000)


# Regularization

what is L1 and L2?
A regression model that uses L1 regularization technique is called Lasso Regression and model which uses L2 is called Ridge Regression.
    

#### First try different c with the L2 penalty (ridge)

In [25]:
# model_regularized=linear_model.LogisticRegression(penalty='l2',C=1/10.24,solver='sag')
# model_regularized.fit(X_train,y_train)
# Y_predict=model_regularized.predict(X_test1)
lamda=[0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
listoferror=[]
for i in lamda:
    lr_reg=LogisticRegression( C=i)
    lr_reg.fit(X_train,y_train)
    pred_val_reg=lr_reg.predict(X_cv)
    accuracy_val=accuracy_score(y_cv,pred_val_reg)
    err_cv=1-accuracy_val
    listoferror.append(1-accuracy_val)
listoferror


C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

[0.0010938889954138498,
 0.0010459527678848834,
 0.0009681546281249398,
 0.0009155033618226671,
 0.0008675671342937008,
 0.0008306326639025974,
 0.0007999849118759395,
 0.0007701229996448511,
 0.0007512628445514036,
 0.0007284734904803303,
 0.0007135425343647306]

In [26]:
lamda[np.argmin(listoferror)]


10.24

#### First try different c with the L1 penalty (Lasso)

In [28]:
lamda1=[0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
listoferror1=[]
for i in lamda1:
    lr_reg=LogisticRegression(penalty='l1', C=i, solver='saga')
    lr_reg.fit(X_train,y_train)
    pred_val_reg=lr_reg.predict(X_cv)
    accuracy_val=accuracy_score(y_cv,pred_val_reg)
    err_cv=1-accuracy_val
    listoferror1.append(1-accuracy_val)
listoferror1

C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Mohamed\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached 

[0.0012361259984094985,
 0.0012329826392272203,
 0.0012329826392272203,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507,
 0.0012321967994316507]

In [29]:
lamda1[np.argmin(listoferror1)]

0.08

In [ ]:
# mtp.scatter( range(0,1272524),y_test1
#            )
# mtp.show()


In [ ]:
# mtp.scatter( range(0,1272524),Y_predict
#            )
# mtp.show()
